In [1]:
from typing import TypedDict

class UserQueryState(TypedDict):
    user_query: str
    response: str
    task: str  # "rag", "math", "news"
    iteration: int


In [45]:
from langchain_groq import ChatGroq

rag_llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0.0
)


In [47]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings  # or Groq embeddings if available
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load PDF
loader = PyPDFLoader("C:/Users/SriHarsha/Desktop/POC/Q2. LG_Multi_model/home_loan.pdf")
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
# Create vector store
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)



C:\Users\SriHarsha\AppData\Local\Temp\ipykernel_7348\4065827976.py:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [49]:
def rag_agent(state: UserQueryState) -> UserQueryState:
    # Retrieve relevant chunks
    query = state["user_query"]
    relevant_docs = vectorstore.similarity_search(query, k=3)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    
    prompt = f"""
    You are a document-based Q&A agent. Use the following context to answer the user question:
    Context:
    {context}

    Question: {query}
    
    Answer clearly and concisely.
    """
    answer = rag_llm.invoke(prompt).content
    return {**state, "response": answer, "iteration": state["iteration"] + 1}

In [50]:
# def rag_agent(state: UserQueryState) -> UserQueryState:
#     prompt = f"""
#     You are a document-based Q&A agent. Use the uploaded document to answer the user question:
    
#     Question: {state['user_query']}
    
#     Answer clearly and concisely.
#     """
#     answer = rag_llm.invoke(prompt).content
#     return {
#         **state,
#         "response": answer,
#         "iteration": state["iteration"] + 1
#     }


In [51]:
from langchain_openai import ChatOpenAI
import os

math_llm = ChatOpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ["NVIDIA_API_KEY"],
    model="meta/llama3-8b-instruct",
    temperature=0
)


In [52]:
def math_agent(state: UserQueryState) -> UserQueryState:
    prompt = f"""
    Solve the following mathematical problem step by step:

    Problem: {state['user_query']}

    Provide the final answer clearly.
    """
    result = math_llm.invoke(prompt).content
    return {
        **state,
        "response": result,
        "iteration": state["iteration"] + 1
    }


In [53]:
from langchain_google_genai import ChatGoogleGenerativeAI

news_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3
)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [54]:
def news_agent(state: UserQueryState) -> UserQueryState:
    prompt = f"""
    Provide the latest news updates on the following topic:

    Topic: {state['user_query']}

    Summarize key points concisely.
    """
    news = news_llm.invoke(prompt).content
    return {
        **state,
        "response": news,
        "iteration": state["iteration"] + 1
    }


In [55]:
classifier_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [56]:
def supervisor_decision_model(state: UserQueryState) -> UserQueryState:
    """
    Uses LLM to decide which agent should handle the query.
    Updates 'task' in the state and returns the full state.
    """
    prompt = f"""
    You are a routing assistant. The user query is:

    "{state['user_query']}"

    Decide which agent should handle this query: 
    Options: "rag_agent", "math_agent", "news_agent".
    Respond with ONLY the agent name.
    """
    agent_choice = classifier_llm.invoke(prompt).content.strip()

    state["task"] = agent_choice
    return state  # Must return a dict, not a string


In [57]:
from langgraph.graph import StateGraph, END

graph = StateGraph(UserQueryState)
graph.add_node("rag_agent", rag_agent)
graph.add_node("math_agent", math_agent)
graph.add_node("news_agent", news_agent)
graph.add_node("supervisor", supervisor_decision_model)


In [58]:
graph.set_entry_point("supervisor")
graph.add_conditional_edges(
    "supervisor",
    lambda state: state["task"],  
    {
        "rag_agent": "rag_agent",
        "math_agent": "math_agent",
        "news_agent": "news_agent",
        "end": END
    }
)
graph.add_edge("rag_agent", END)
graph.add_edge("math_agent", END)
graph.add_edge("news_agent", END)

multi_agent_graph = graph.compile()

In [61]:
user_query = input("\nEnter your query: ")
initial_state = {
    "user_query": user_query,
    "response": "",
    "task": "",
    "iteration": 0
}
final_state = multi_agent_graph.invoke(initial_state)
print("\n✅ RESPONSE FROM AGENT\n")
print(final_state["response"])
print(f"Task executed: {final_state['task']}")
print(f"Iterations: {final_state['iteration']}")


✅ RESPONSE FROM AGENT

The LTV ratio for a home loan above ₹75 lacs is **75 %**.  
Multiplying by 3 gives **225 %**.
Task executed: rag_agent
Iterations: 1
